In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd
from google.colab import drive
import gc
import plotly.graph_objects as go

In [ ]:
drive.mount('.../drive')

Mounted at /content/drive


In [ ]:
urltt1 = '..1.csv'
urltt2 = '..2.csv'

In [ ]:
df1 = pd.read_csv(urltt1)
df2 = pd.read_csv(urltt2)

In [ ]:
gc.collect()

60

In [ ]:
ddf1 = dd.from_pandas(df1, npartitions=10)
ddf2 = dd.from_pandas(df2, npartitions=10)

ddf1['date'] = dd.to_datetime(ddf1['date'])
ddf2['date'] = dd.to_datetime(ddf2['date'])

merged_ddf = ddf1.merge(ddf2, on=['global_user_id', 'date'], how='left')

merged_ddf.to_csv('data.csv', mode='a', index=False)

['/content/m10.csv']

In [ ]:
!cp data.csv /content...

In [ ]:
murl = '.../data.csv'
mdf = pd.read_csv(murl)

In [ ]:
nan_counts = mdf.isna().sum()
print('Number of NaN values in each column of merged dataframe:')
print(nan_counts)

In [ ]:
ad_users = mdf[~mdf['reason'].isna()]
non_ad_users = mdf[mdf['reason'].isna()]
ad_users.loc[:, 'is_special_ad'] = ad_users['reason'].str.contains('-', na=False)
special_ad_count = ad_users['is_special_ad'].value_counts()[True]
regular_count = ad_users['is_special_ad'].value_counts()[False]

print('Number of special_ad users:', special_ad_count)
print('Number of regular ad users:', regular_count)

In [ ]:
nan_counts = ad_users.isna().sum()
print(nan_counts)

In [ ]:
groups = ad_users.groupby('is_special_ad')

average_revenue = groups['net_revenue_usd_iap_daily'].mean()
average_playtime = groups['playtime_daily'].mean()
average_session_count = groups['session_count_daily'].mean()
average_transaction_count = groups['iap_transaction_count_daily'].mean()

print('Average revenue by special_ad watchers:', average_revenue[True])
print('Average revenue by regular-ad watchers:', average_revenue[False])
print('Average daily playtime by special_ad watchers:', average_playtime[True])
print('Average daily playtime by regular-ad watchers:', average_playtime[False])
print('Average daily session count by special_ad watchers:', average_session_count[True])
print('Average daily session count by regular-ad watchers:', average_session_count[False])
print('Average daily transaction count by special_ad watchers:', average_transaction_count[True])
print('Average daily transaction count by regular-ad watchers:', average_transaction_count[False])

Average revenue by special_ad watchers: 0.09577250166108613
Average revenue by regular-ad watchers: 0.08856478860761281
Average daily playtime by special_ad watchers: 5299164.458462905
Average daily playtime by regular-ad watchers: 5403467.616388043
Average daily session count by special_ad watchers: 9.196357174589071
Average daily session count by regular-ad watchers: 9.079482923131216
Average daily transaction count by special_ad watchers: 0.01776988005330964
Average daily transaction count by regular-ad watchers: 0.020259584090344673


In [ ]:
groups = ad_users.groupby('is_special_ad')
average_iap_revenue = groups['net_revenue_usd_iap_daily'].mean()
average_ad_revenue = groups['net_revenue'].mean()
average_playtime = groups['playtime_daily'].mean()
average_session_count = groups['session_count_daily'].mean()
average_transaction_count = groups['iap_transaction_count_daily'].mean()

metrics = ['Daily Net Revenue from in Game Purchases', 'Net Revenue from Ads', 'Daily Playtime in miliseconds', 'Daily Session Count', 'Daily Transaction Count']

mean_values_special_ad = [average_iap_revenue[True], average_ad_revenue[True], average_playtime[True], average_session_count[True], average_transaction_count[True]]
mean_values_regular = [average_iap_revenue[False], average_ad_revenue[False], average_playtime[False], average_session_count[False], average_transaction_count[False]]

for i in range(len(metrics)):
    fig = go.Figure(data=[
        go.Bar(name='Special-ad watchers', x=['Special ad'], y=[mean_values_special_ad[i]]),
        go.Bar(name='Regular-ad watchers', x=['Regular'], y=[mean_values_regular[i]])
    ])
    fig.update_layout(barmode='group', title_text=f'Average {metrics[i]} by special-ad watchers and regular-ad watchers')

    fig.show(config={'scrollZoom': True})

In [ ]:
cross_tab = pd.crosstab(ad_users['is_special_ad'], ad_users['is_payer'])
print(cross_tab)

is_payer_lifetime  False  True 
is_special_ad                  
False              61535  14972
True                5246   1507


In [ ]:
cross_tab = pd.crosstab(ad_users['is_special_ad'], ad_users['is_payer'])

fig = go.Figure()

fig.add_trace(go.Bar(
    x=['Regular ad', 'Special ad'],
    y=cross_tab[True],
    name='Made purchase in the game',
    marker_color='blue'
))

fig.add_trace(go.Bar(
    x=['Regular ad', 'Special ad'],
    y=cross_tab[False],
    name='Did not make purchase in the game',
    marker_color='red'
))

fig.update_layout(barmode='stack', title_text='Purchase Behavior Comparison: Special ad users vs. Regular ad users')

fig.update_layout(
    legend=dict(
        title='is_payer',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)


fig.show()

In [ ]:
ad_users_filtered = ad_users[ad_users['is_payer'] == False]

special_ad_watchers = ad_users_filtered[ad_users_filtered['is_special_ad'] == True]

regular_watchers = ad_users_filtered[ad_users_filtered['is_special_ad'] == False]

average_special_ad_revenue = special_ad_watchers['net_revenue'].mean()
average_regular_revenue = regular_watchers['net_revenue'].mean()

fig = go.Figure(data=[
   go.Bar(name='Special ad', x=['Special ad'], y=[average_special_ad_revenue]),
   go.Bar(name='Regular ad', x=['Regular ad'], y=[average_regular_revenue])
])
fig.update_layout(barmode='group', title_text='Average Net Revenue from ads (special-ad non-payers vs regular-ad non-payers')

fig.show(config={'scrollZoom': True})
